In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime, openpyxl
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
tsql="""
		IF ISNUMERIC(REPLACE(@M1,'.',''))<>1
		BEGIN
			SET @M1=0 
		END

		If NOT EXISTS(SELECT * FROM fsma_Answers WHERE FK_OrgSubmission=@PK_OrgSubmission AND FK_Question=22402)
		BEGIN
			INSERT INTO fsma_Answers (FK_Question, FK_OrgSubmission, DateModifed) VALUES (22402, @PK_OrgSubmission, @DateImported)
		END

		INSERT INTO AuditLog (TableName, PK_PrimeKey, FieldName, FieldValue, Change_Date, UserID, PK_Agency, EditType) 
		VALUES ('fsma_Answers', @PK_OrgSubmission, '22402', (SELECT TOP 1 Answer FROM fsma_Answers WHERE FK_OrgSubmission=@PK_OrgSubmission AND FK_Question=22402), @DateImported, @UserID, -1, 'U')

		UPDATE A SET Answer=@M1
		FROM fsma_Answers A
		INNER JOIN vw_OrgSubQuestions Q  ON Q.PK_Question=A.FK_Question AND Q.PK_OrgSubmission=@PK_OrgSubmission 
		WHERE A.FK_OrgSubmission=@PK_OrgSubmission
		AND Q.PK_Question=22402
        """


df=sql_todf("""
	SELECT PK_Question, identifier_text
	FROM  vwOrgSubToComponent ORG 
	INNER JOIN vw_OrgSubQuestions Q  ON ORG.PK_OrgSubmission =Q.PK_OrgSubmission 
	LEFT JOIN fsma_Answers A ON ORG.PK_OrgSubmission=A.FK_OrgSubmission 
	WHERE ORG.PK_OrgSubmission=26445 AND PK_QUESTION NOT IN (22401,22407)
""", connstr)   
df['m']=df['identifier_text'].apply( lambda s: f'@M{s.replace(".","_")}' )
df['tsql'] = [ tsql.replace('@M1',x[0] ).replace('22402',str(x[1]) ) for x in zip(df['m'], df['PK_Question'])]
for s in df['tsql']:
    print(s)

In [ ]:
df, code = generate_code_from_db(ctx, 3273)
df[21:25]
#print(code)

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df[12:14])
print(code)

In [ ]:
df, code = generate_code_from_db(ctx, 3273)
df

In [ ]:
def hvasa_pages():
    df=sql_todf("""
    SELECT DISTINCT PK_FORM, PK_QuestionGroup, SectionNum, ASPX
    FROM vwQuestions
    WHERE PK_FORM='2023-A-HVAENDPRO'
    ORDER BY PK_QuestionGroup ASC
    """, connstr)   
    df['FASPX']=df['ASPX'].str.replace('HVA/2023/','')
    df['PNAME']=df['FASPX'].str.replace('.aspx','') 
    pages=df.to_dict(orient='records') 
    #print(pages)
    #return 
    base = r"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\HVA\\2023\\" 
    src = r'C:\\_som\\tmp\\'
    for p in pages:  
        for ext in ['.aspx','.aspx.designer.vb','.aspx.vb']:
            with open(f'{src}2023_HVA_ENDPOINT_1{ext}', 'r', encoding='UTF-8') as fr: 
                txt = fr.read()
                txt=txt.replace(f'11',f"{p['SectionNum']}{p['SectionNum']}")
                txt=txt.replace(f"INT_1",f"INT_{p['SectionNum']}")
                txt=txt.replace('{PK_key}',f"{p['PK_QuestionGroup']}")
                #txt=txt.replace('ENDPOINT',f"REMOTE")
                if ext=='.aspx': 
                    df, code = generate_code_from_db(ctx, qgroup=p['PK_QuestionGroup'])
                    txt=txt.replace(f'<!--controls-->',code)
                #print(txt)
            with open(f"{base}{p['PNAME']}{ext}", 'w', encoding='UTF-8') as fw:
                fw.write(txt)

hvasa_pages()

In [ ]:
df=sql_todf("""
SELECT ORG.Acronym Agency, PK_CISA_CVE [CVE ID], RIGHT(PK_CISA_CVE, 4) [Count]
FROM CVEs
OUTER APPLY(
	SELECT PK_OrgSubmission, PK_Component, Acronym FROM vwOrgSubToComponent 
    WHERE PK_ReportingCycle=108 AND OrgSub_Description='ACTIVE' AND Acronym='USDA'
) ORG

""", connstr)  
df.to_csv(r'C:\Users\timko\Downloads\usda.csv', index=False)
df

In [16]:
ip = issue_provider(ctx)  
#ip.driver.quit()
#ip.issues
ip.parse_metrics(tnum='9495')
#print(ip.issues.to_string())



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST driver version for 111.0.5563
There is no [win32] chromedriver for browser 111.0.5563 in cache
Get LATEST driver version for 111.0.5563
Trying to download new driver from https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\timko\.wdm\drivers\chromedriver\win32\111.0.5563.64]


        Key                                                                                   Summary
0   CS-9496                               PQC Section 1 - remove limit on 'Budget Estimate' questions
1   CS-9495                                           PQC Section 1- update required/optional columns
2   CS-9494                                 PQC Section 1 - add 1 to row number on upload validations
3   CS-9493                          Disable PQC - Section 1 Upload buttons when the form is 'Locked'
4   CS-9484                          Create BOD 20-01 VDP Maintenance Upload to populate agency forms
5   CS-9261                                       BOD 18-02 Annual 2023: Manage HVAs Agency Interface
6   CS-8885                                                      create data request/response service
7   CS-8678                       BOD 18-02 Annual HVA 2021: Update 'Manage HVA Interface' Audit Logs
8   CS-8459                                                       Selenium Browser

In [ ]:
ip.driver.quit()

In [ ]:
print(ip.issues.to_string())

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '2.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 

In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    ip = issue_provider(ctx)
    driver=ip.driver
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read().replace('\n','')

def met_provider(s):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)  
    return m[0] if len(m) > 0 else ''

def cell_provider(s, i):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}' if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4014", ctx.config['connstr'])
    print(dff.iloc[0])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids()   
list(ids.keys())[3:5] 
 

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 

#df.fillna('', inplace=True)
#df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df.loc[1:,c] = df.loc[1:, 'Question'].apply(lambda s:  cell_provider(s,i)  )

#mets = [str(m)  for m in list(dff['m']) if '2' in str(m) ]  

df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
 
s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('th>','td>',s) 
s=re.sub('<td>Number','<td colspan="2">Number',s)

s=re.sub('<thead>|</thead>|<tbody>|</tbody>','',s) 
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f: 
    s=html.unescape(s) 
    f.write(s)
df[:5]

In [ ]:
def list_to_sql(items=[], PK_PickListType=808, PK_PickList=9999, Description='[Description]', UsageField='[UsageField]', encoding=None):
    if encoding == None:
        encoding=lambda s: re.sub('[^A-Z0-9]','',s.upper().strip()[:15])  
    plt=''
    with open(f'{ctx.get_tempalte_dir()}plt.sql', 'r') as f:
        plt=f.read()
        plt=plt.replace('{PK_PickListType}',str(PK_PickListType))
        plt=plt.replace('{Description}',Description)
        plt=plt.replace('{UsageField}',UsageField)
    pl=''
    with open(f'{ctx.get_tempalte_dir()}picklists.sql', 'r') as f:
        pl=f.read()
    sql_items=[] 
    for i, item in enumerate(list(items)): 
        s=pl.replace('{PK_PickList}',str(PK_PickList+i))
        s=s.replace('{CodeValue}',encoding(item))
        s=s.replace('{DisplayValue}',str(item)) 
        s=s.replace('{SORT}',str(i+1)) 
        sql_items.append(s)
    PK_PickList=PK_PickList+i+10
    sql=plt.replace('{picklists.sql}','\n,'.join(sql_items)[:])
    return sql_items, sql
l,sql = list_to_sql(['asdf asdf', '098a asdfasdfasdfasdfasdfasdfasdf @#$%'], Description='Desc', UsageField='UF')
print(len(l))
print(sql)